In [18]:
from enum import Enum

from PIL import ImageOps, Image
from tqdm import tqdm
import numpy as np

import glob


class Dataset(Enum):
    TRAIN = "../datasets/chest_Xray/train/"
    TEST = "../datasets/chest_Xray/test/"
    VAL = "../datasets/chest_Xray/val/"


class Values(Enum):
    NORMAL = 0
    PNEUMONIA = 1


def import_dataset(dataset_type, img_length):
    print("Loading dataset " + dataset_type.value)
    images = []
    for rayX_type in Values:
        new_images = glob.glob(dataset_type.value + rayX_type.name + '/*.jpeg')
        images += new_images
    dataset_x = [[[None]*img_length]*img_length]*len(images)
    dataset_y = [None]*len(images)
    for i in tqdm(range(len(images)), desc="Importing images", colour="green"):
        img = Image.open(images[i])
        if img.mode != "L":
            img = ImageOps.grayscale(img)
        resized_img = ImageOps.pad(img, (img_length, img_length))
        arr_img = np.asarray(resized_img)
        dataset_x[i] = arr_img
        if "pneumonia" in images[i].lower():
            dataset_y[i] = 1
        elif "normal" in images[i].lower():
            dataset_y[i] = 0

    print("Returning processed data")
    dataset_x = np.array(dataset_x)
    dataset_y = np.array(dataset_y)
    return dataset_x, dataset_y


def format_dataset(data_x, img_length):
    print("Reshaping data to 1D array")
    data_x = data_x.reshape(len(data_x), img_length * img_length)
    print("Changing type to float")
    data_x = data_x.astype('float32')
    print("Normalize data")
    data_x /= 255
    return data_x


In [19]:
# import pickle
import pickle

import numpy as np
from keras import Sequential, Model
from keras.applications import VGG19
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras import utils as np_utils
from keras.models import save_model, load_model
from sklearn.neural_network import MLPClassifier

import dataset

img_length = 750

# vgg model
# img_length = 32
#

#use pickle but error with Adam
def get_ai(regenerate=True):
    # if regenerate == True:
    #     model = start_training()
    #     save_ai(model)
    #     return model
    try:
        with open("model.pkl", "rb") as f:
            print("Found an existing IA")
            model = pickle.load(f)
            start_with_data(model)
            return model

    except Exception as ex:
        print("Error IA not found : ", str(ex))
        model = start_training()
        save_ai(model)
        return model

def save_ai(ai):
    try:
        with open("model.pkl", "wb") as f:
            pickle.dump(ai, f, protocol=pickle.HIGHEST_PROTOCOL)
            print("IA model create")
    except Exception as ex:
        print("Error during pickling object (Possibly unsupported):", ex)

##

#save and load model without pickle
# def get_ai(regenerate=True):
#     # if regenerate == True:
#     #     model = start_training()
#     #     save_ai(model)
#     #     return model
#     try:
#         with open("model.keras", "rb") as f:
#             print("Found an existing IA")
#             model = load_model("model.keras")
#             start_with_data(model)
#             return model
#
#     except Exception as ex:
#         print("Error IA not found : ", str(ex))
#         model = start_training()
#         save_ai(model)
#         return model
#
# def save_ai(ai):
#     try:
#         ai.save("model.keras")
#         print("Modèle IA créé et sauvegardé.")
#     except Exception as ex:
#         print("Erreur lors de la sauvegarde du modèle IA :", str(ex))

##

def start_training():
    print("Regenerating IA")
    train_x, train_y = import_dataset(Dataset.TRAIN, img_length)
    train_x = format_dataset(train_x, img_length)
    train_y = np_utils.to_categorical(train_y, 2)

    test_x, test_y = import_dataset(dataset.Dataset.TEST, img_length)
    test_x = format_dataset(test_x, img_length)
    test_y = np_utils.to_categorical(test_y, 2)

    #choose your model
    model = mlp_model(train_x, train_y, test_x, test_y)

    return model

def start_with_data(model):

    test_x, test_y = import_dataset(Dataset.TEST, img_length)
    test_x = format_dataset(test_x, img_length)
    test_y = np_utils.to_categorical(test_y, 2)

    score(model, test_x, test_y)





def cnn_model(train_x, train_y, test_x, test_y):
    # building a linear stack of layers with the sequential model
    model = Sequential()
    # hidden layer
    model.add(Dense(100, input_shape=(img_length * img_length,), activation='relu'))
    # output layer
    model.add(Dense(2, activation='softmax'))

    # looking at the model summary
    model.summary()
    # compiling the sequential model
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    # training the model for 10 epochs
    model.fit(train_x, train_y, batch_size=128, epochs=10, validation_data=(test_x, test_y))
    score(model, test_x, test_y)

    # Calculer la précision du modèle sur l'ensemble de test
    test_acc = model.evaluate(test_x, test_y)[1]
    print('Précision sur l\'ensemble de test :', test_acc)

    return model

def mlp_model(train_x, train_y, test_x, test_y):
    # building a linear stack of layers with the sequential model
    model = MLPClassifier(hidden_layer_sizes=(300,), verbose=2)

    model.fit(train_x, train_y)

    score(model, test_x, test_y)

    # Calculer la précision du modèle sur l'ensemble de test
    test_acc = model.score(test_x, test_y)
    print('Précision sur l\'ensemble de test :', test_acc)

    return model

def vgg_model(list_x, list_y, test_x, test_Y):

    print('X (vecteur): vgg:  ' + str(list_x.shape))


    #Redimonsionner en 32\
    list_x= np.repeat(list_x[..., np.newaxis], 3, axis=-1)
    list_x = np.array(list_x).reshape(-1, img_length, img_length, 3)
    print('X (vecteur) vgg: ' + str(list_x.shape))

    test_x= np.repeat(test_x[..., np.newaxis], 3, axis=-1)
    test_x = np.array(test_x).reshape(-1, img_length, img_length, 3)


    print('X (vecteur) vgg: ' + str(test_x.shape))


    # Charger le modèle VGG19 pré-entraîné
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_length, img_length, 3))

    # Geler les poids des couches du modèle VGG19
    for layer in base_model.layers:
        layer.trainable = False

    # Ajouter les nouvelles couches de classification
    x = base_model.output
    x = Flatten()(x)
    x = Dense(300, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)

    # Créer le modèle final en spécifiant les entrées et les sorties
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compiler le modèle avec une fonction de perte de catégorie croisée, un optimiseur Adam et la métrique de précision
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

    # Entraîner le modèle en utilisant les images d'entraînement et les étiquettes correspondantes
    model.fit(list_x, list_y, batch_size=128, epochs=10, validation_data=(test_x, test_Y))

    score(model, test_x, test_Y)

    # Évaluer le modèle sur l'ensemble de test
    test_loss, test_acc = model.evaluate(test_x, test_Y)
    print('Perte sur l\'ensemble de test :', test_loss)
    print('Précision sur l\'ensemble de test :', test_acc)


    return model


def score(model, test_x, test_y):
    test_y_pred = np.argmax(model.predict(test_x), axis=-1)
    test_y_inverse = np.argmax(test_y, axis=1)
    print('L\'ensemble de test :\n', test_y_inverse)
    print('Prédictions sur l\'ensemble de test :\n', test_y_pred)
    # print("\n")
    print('20 premieres données de test :\n', test_y_inverse[:20])
    print('20 premieres données de la prediction de test :\n', test_y_pred[:20])

    # # Calculer la précision du modèle sur l'ensemble de test
    # test_acc = model.evaluate(test_x, test_y)[1]
    # print('Précision sur l\'ensemble de test :', test_acc)


In [20]:
ai = get_ai(True)

Found an existing IA
Loading dataset ../datasets/chest_Xray/test/


Importing images: 100%|██████████| 624/624 [00:05<00:00, 116.70it/s]


Returning processed data
Reshaping data to 1D array
Changing type to float
Normalize data
L'ensemble de test :
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 